<a href="https://colab.research.google.com/github/Yakagi17/Transfer_Learning_FGA_UII/blob/master/%5B31_5%5D_Transfer_Learning_CIFAR_10_ADF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src = "https://i.imgur.com/UjutVJd.jpg" align = "center">

# Transfer Learning
Di sini kita akan mencoba melatih VGG16 untuk mengklasifikasikan dataset CIFAR-10

Kema yang akan kita coba antara lain:
* Melatih VGG16 dari awal (bobot random)
* Melatih VGG16 dari awal dengan Data Augmentasi
* Melatih keseluruhan pretrained VGG16 (sudah dilatih menggunakan ImageNet)
* Melatih hanya Layer FC dari pretrained VGG16
* Melatih beberapa layer dari pretrained VGG16



In [0]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
%matplotlib inline

np.set_printoptions(precision=7)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras import Model
from tensorflow.keras.layers import Input

from tensorflow.keras.utils import to_categorical
from tensorflow.keras import backend as K

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image


# CIFAR-10 Dataset

## Load CIFAR-10 Dataset

In [0]:
from tensorflow.keras.datasets import cifar10

(X_train, y_train), (X_test, y_test) = cifar10.load_data()

print('X_train.shape =',X_train.shape)
print('y_train.shape =',y_train.shape)
print('X_test.shape  =',X_test.shape)
print('y_test.shape  =',y_test.shape)

X_val = X_train[-10000:,:]
y_val = y_train[-10000:]
X_train = X_train[:-10000, :]
y_train = y_train[:-10000]

print('X_val.shape   =',X_val.shape)
print('y_val.shape   =',y_val.shape)
print('X_train.shape =',X_train.shape)
print('y_train.shape =',y_train.shape)

## Normalize Images

In [0]:
X_train = X_train.astype('float32')
X_val = X_val.astype('float32')
X_test = X_test.astype('float32')

mean_image = np.mean(X_train, axis = 0)
X_train -= mean_image
X_val -= mean_image
X_test -= mean_image

y_train_hot = to_categorical(y_train)
y_val_hot = to_categorical(y_val)
y_test_hot = to_categorical(y_test)

print('y_train_hot.shape =',y_train_hot.shape)
print('y_val_hot.shape   =',y_val_hot.shape)
print('y_test_hot.shape  =',y_test_hot.shape)

## Define Input-output shape

In [0]:
input_shape = X_train.shape[1:]
num_class = 10

batch_size = 512
epochs = 10
opt = tf.keras.optimizers.Adam(lr=0.0001, decay=1e-6)

# Custom VGG16 for Cifar-10

Untuk kasus ini, kita akan gunakan arsitektur VGG16 dengan input berukuran (32, 32, 3) sesuai inpuut data Cifar-10

Untuk kepala output, setelah MaxPool terakhir, kita akan gunakan tambahkan 3 layer:
* `Global Average Pooling`
* `Dense(1024, 'relu')`
* `Dense(10, 'softmax')`

<br>

Penambahan layer `GlobalAveragePooling2D` bisa digantikan dengan menambahkan parameter `pooling='avg'` pada saat mengunduh model VGG16 dengan `include_top=False`

# Train VGG16 from Scratch

In [0]:
model = VGG16(weights=None, include_top=False, input_shape=input_shape, pooling='avg')

x = model.output
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_class, activation='softmax')(x)

myModel_scratch = Model(inputs=model.input, outputs=predictions, name='VGG Scratch')
myModel_scratch.summary()

Sekarang mari kita latih model pertama kita CIFAR-10

In [0]:
myModel_scratch.compile(loss='categorical_crossentropy',
                        optimizer=opt, metrics=['accuracy'])

In [0]:
hist_scratch = myModel_scratch.fit(X_train, y_train_hot,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_data=(X_val, y_val_hot),
                    shuffle=True)

In [0]:
plt.rcParams['figure.figsize'] = [7, 5]

plt.plot(hist_scratch.history['acc'], label='train')
plt.plot(hist_scratch.history['val_acc'], label='val')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [0]:
scores_scratch = myModel_scratch.evaluate(X_test, y_test_hot, verbose=1)

In [0]:
print('Test loss    :', scores_scratch[0])
print('Test accuracy: %.2f%%' % (scores_scratch[1]*100))

Dapat dilihat, melatih VGG16 pada dataset CIFAR-10 membuat overfit

# Train VGG16 from Scratch with Data Autmentation

In [0]:
model = VGG16(weights=None, include_top=False, input_shape=input_shape, pooling='avg')

x = model.output
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_class, activation='softmax')(x)

myModel_aug = Model(inputs=model.input, outputs=predictions, name='VGG Augmented')
myModel_aug.summary()

Define Data Autmentation schedule

In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=False)

Perform Data Augmentation

In [0]:
datagen.fit(X_train)

In [0]:
myModel_aug.compile(loss='categorical_crossentropy',
                        optimizer=opt, metrics=['accuracy'])

hist_aug = myModel_aug.fit_generator(datagen.flow(X_train, y_train_hot,
                                             batch_size=batch_size),
                                epochs=epochs,
                                validation_data=(X_val, y_val_hot))

In [0]:
plt.rcParams['figure.figsize'] = [7, 5]

plt.plot(hist_aug.history['acc'], label='train')
plt.plot(hist_aug.history['val_acc'], label='val')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


In [0]:
plt.rcParams['figure.figsize'] = [15, 5]

plt.subplot(1, 2, 1)
plt.plot(hist_scratch.history['acc'], label='scratch')
plt.plot(hist_aug.history['acc'], label='aug')
plt.xlabel('Epoch')
plt.ylabel('Train Acc')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(hist_scratch.history['val_acc'], label='scratch')
plt.plot(hist_aug.history['val_acc'], label='aug')
plt.xlabel('Epoch')
plt.ylabel('Val Acc')
plt.legend()
plt.show()



In [0]:
scores_aug = myModel_aug.evaluate(X_test, y_test_hot, verbose=1)

In [0]:
print('Test loss    :', scores_aug[0])
print('Test accuracy: %.2f%%' % (scores_aug[1]*100))

Dapat dilihat, Augmentasi Data dapat memperkecil overfitting

# Train ImageNet-pretrained VGG16

Load model pretrained VGG without the top FC layer,

this code below will download new model without the top FC Layer

In [0]:
model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape, pooling='avg')

x = model.output
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_class, activation='softmax')(x)

myModel_imagenet = Model(inputs=model.input, outputs=predictions, name='VGG ImageNet')
myModel_imagenet.summary()

In [0]:
myModel_imagenet.compile(loss='categorical_crossentropy',
                        optimizer=opt, metrics=['accuracy'])

In [0]:
hist_imagenet = myModel_imagenet.fit(X_train, y_train_hot,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_data=(X_val, y_val_hot),
                    shuffle=True)

In [0]:
plt.rcParams['figure.figsize'] = [7, 5]

plt.plot(hist_imagenet.history['acc'], label='train')
plt.plot(hist_imagenet.history['val_acc'], label='val')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


In [0]:
plt.rcParams['figure.figsize'] = [15, 5]

plt.subplot(1, 2, 1)
plt.plot(hist_scratch.history['acc'], label='scratch')
plt.plot(hist_aug.history['acc'], label='aug')
plt.plot(hist_imagenet.history['acc'], label='imagenet')
plt.xlabel('Epoch')
plt.ylabel('Train Acc')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(hist_scratch.history['val_acc'], label='scratch')
plt.plot(hist_aug.history['val_acc'], label='aug')
plt.plot(hist_imagenet.history['val_acc'], label='imagenet')
plt.xlabel('Epoch')
plt.ylabel('Val Acc')
plt.legend()
plt.show()



In [0]:
scores_imagenet = myModel_imagenet.evaluate(X_test, y_test_hot, verbose=1)

In [0]:
print('Test loss    :', scores_imagenet[0])
print('Test accuracy: %.2f%%' % (scores_imagenet[1]*100))

Penggunaan Pretrained Model membuat model belajar lebih cepat, dengan masih menjaga dari overfitting meskipun tanpa Augmentasi Data 

# Train FC layer of ImageNet-pretrained VGG16

Load model pretrained VGG without the top FC layer,

In [0]:
model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape, pooling='avg')

Let's freeze the pretrained model so it won't be changed during training

In [0]:
for layer in model.layers:
    layer.trainable = False    

Add new layers

In [0]:
x = model.output
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_class, activation='softmax')(x)

myModel_fc = Model(inputs=model.input, outputs=predictions, name='VGG FC')
myModel_fc.summary()

In [0]:
myModel_fc.compile(loss='categorical_crossentropy',
                        optimizer=opt, metrics=['accuracy'])

In [0]:
hist_fc = myModel_fc.fit(X_train, y_train_hot,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_data=(X_val, y_val_hot),
                    shuffle=True)

In [0]:
plt.rcParams['figure.figsize'] = [7, 5]

plt.plot(hist_fc.history['acc'], label='train')
plt.plot(hist_fc.history['val_acc'], label='val')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


In [0]:
plt.rcParams['figure.figsize'] = [15, 5]

plt.subplot(1, 2, 1)
plt.plot(hist_scratch.history['acc'], label='scratch')
plt.plot(hist_aug.history['acc'], label='aug')
plt.plot(hist_imagenet.history['acc'], label='imagenet')
plt.plot(hist_fc.history['acc'], label='fc')
plt.xlabel('Epoch')
plt.ylabel('Train Acc')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(hist_scratch.history['val_acc'], label='scratch')
plt.plot(hist_aug.history['val_acc'], label='aug')
plt.plot(hist_imagenet.history['val_acc'], label='imagenet')
plt.plot(hist_fc.history['val_acc'], label='fc')
plt.xlabel('Epoch')
plt.ylabel('Val Acc')
plt.legend()
plt.show()



In [0]:
scores_fc = myModel_fc.evaluate(X_test, y_test_hot, verbose=1)

In [0]:
print('Test loss    :', scores_fc[0])
print('Test accuracy: %.2f%%' % (scores_fc[1]*100))

# Train Selected Layers

we can also choose which layer to train and which layer to freeze

First let's see the layer ID and name

In [0]:
model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape, pooling='avg')

x = model.output
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_class, activation='softmax')(x)

myModel_selected = Model(inputs=model.input, outputs=predictions, name='VGG Selected')

In [0]:
for i, layer in enumerate(myModel_selected.layers):
    print(i, layer.name)

Choose which to freeze

In [0]:
for layer in myModel_selected.layers[:16]:
    layer.trainable = False
for layer in myModel_selected.layers[16:]:
    layer.trainable = True
    
myModel_selected.summary()

In [0]:
myModel_selected.compile(loss='categorical_crossentropy',
                        optimizer=opt, metrics=['accuracy'])


In [0]:
hist_selected = myModel_selected.fit(X_train, y_train_hot,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_data=(X_val, y_val_hot),
                    shuffle=True)

In [0]:
plt.rcParams['figure.figsize'] = [7, 5]

plt.plot(hist_selected.history['acc'], label='train')
plt.plot(hist_selected.history['val_acc'], label='val')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


In [0]:
plt.rcParams['figure.figsize'] = [15, 5]

plt.subplot(1, 2, 1)
plt.plot(hist_scratch.history['acc'], label='scratch')
plt.plot(hist_aug.history['acc'], label='aug')
plt.plot(hist_imagenet.history['acc'], label='imagenet')
plt.plot(hist_fc.history['acc'], label='fc')
plt.plot(hist_selected.history['acc'], label='selected')
plt.xlabel('Epoch')
plt.ylabel('Train Acc')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(hist_scratch.history['val_acc'], label='scratch')
plt.plot(hist_aug.history['val_acc'], label='aug')
plt.plot(hist_imagenet.history['val_acc'], label='imagenet')
plt.plot(hist_fc.history['val_acc'], label='fc')
plt.plot(hist_selected.history['val_acc'], label='selected')
plt.xlabel('Epoch')
plt.ylabel('Val Acc')
plt.legend()
plt.show()



In [0]:
scores_selected = myModel_selected.evaluate(X_test, y_test_hot, verbose=1)

In [0]:
print('Test loss    :', scores_selected[0])
print('Test accuracy: %.2f%%' % (scores_selected[1]*100))

# Train Cut Model

In [0]:
model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape, pooling='avg')

for i, layer in enumerate(model.layers):
    print(i, layer.name)

In [0]:
# Extract the last layer from third block of vgg16 model
last = model.get_layer('block4_conv3').output

# model baru
x = GlobalAveragePooling2D()(last)
x = Dense(1024, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)

myModel_cut = Model(inputs=model.input, outputs=predictions, name='VGG Cut')
myModel_cut.summary()

In [0]:
myModel_cut.compile(loss='categorical_crossentropy',
                        optimizer=opt, metrics=['accuracy'])


In [0]:
hist_cut = myModel_cut.fit(X_train, y_train_hot,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_data=(X_val, y_val_hot),
                    shuffle=True)

In [0]:
plt.rcParams['figure.figsize'] = [7, 5]

plt.plot(hist_cut.history['acc'], label='train')
plt.plot(hist_cut.history['val_acc'], label='val')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


In [0]:
plt.rcParams['figure.figsize'] = [15, 5]

plt.subplot(1, 2, 1)
plt.plot(hist_scratch.history['acc'], label='scratch')
plt.plot(hist_aug.history['acc'], label='aug')
plt.plot(hist_imagenet.history['acc'], label='imagenet')
plt.plot(hist_fc.history['acc'], label='fc')
plt.plot(hist_selected.history['acc'], label='selected')
plt.plot(hist_cut.history['acc'], label='cut')
plt.xlabel('Epoch')
plt.ylabel('Train Acc')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(hist_scratch.history['val_acc'], label='scratch')
plt.plot(hist_aug.history['val_acc'], label='aug')
plt.plot(hist_imagenet.history['val_acc'], label='imagenet')
plt.plot(hist_fc.history['val_acc'], label='fc')
plt.plot(hist_selected.history['val_acc'], label='selected')
plt.plot(hist_cut.history['val_acc'], label='cut')
plt.xlabel('Epoch')
plt.ylabel('Val Acc')
plt.legend()
plt.show()



In [0]:
scores_cut = myModel_cut.evaluate(X_test, y_test_hot, verbose=1)

In [0]:
print('Test loss    :', scores_cut[0])
print('Test accuracy: %.2f%%' % (scores_cut[1]*100))

# Overall Scores

In [0]:
labels = ['scratch', 'augmented', 'imagenet','fc layer','selected','cut']
y_pos = np.arange(len(labels))
test_scores = [scores_scratch[1],scores_aug[1],scores_imagenet[1],scores_fc[1],scores_selected[1],scores_cut[1]]
val_scores = [hist_scratch.history['val_acc'][-1],hist_aug.history['val_acc'][-1],
              hist_imagenet.history['val_acc'][-1],hist_fc.history['val_acc'][-1],
              hist_selected.history['val_acc'][-1],hist_cut.history['val_acc'][-1]]

plt.bar(y_pos, val_scores, .4, color='g',alpha=0.5, label='val acc')
plt.bar(y_pos+.41, test_scores, .4, color='b',alpha=0.5, label='test acc')
plt.xticks(y_pos+.2, labels)
plt.ylabel('Accuracy')
plt.title('Overall Scores')
plt.ylim((.5,.8))
plt.legend()

plt.show()

<p>Copyright &copy; 2019 <a href=https://www.linkedin.com/in/andityaarifianto/>ADF</a> </p>